In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ncaa-men-538-team-ratings/538ratingsMen.csv
/kaggle/input/march-machine-learning-mania-2024/Conferences.csv
/kaggle/input/march-machine-learning-mania-2024/sample_submission.csv
/kaggle/input/march-machine-learning-mania-2024/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2024/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2024/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2024/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2024/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2024/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2024/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2024/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv
/k

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

2024-03-21 06:26:49.591843: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 06:26:49.591910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 06:26:49.593415: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Teams and Seasons

In [3]:
teams_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MTeams.csv")
teams_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WTeams.csv")

seasons_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MSeasons.csv")
seasons_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WSeasons.csv")

### Tournament Seeds and Results

In [4]:
tourney_seeds_2024 = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/2024_tourney_seeds.csv")

tournament_seeds_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySeeds.csv")
tournament_seeds_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WNCAATourneySeeds.csv")

season_results_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MRegularSeasonCompactResults.csv")
season_results_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WRegularSeasonCompactResults.csv")

tournament_results_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyCompactResults.csv")
tournament_results_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WNCAATourneyCompactResults.csv")

### Detailed Game Results

In [5]:
detailed_season_results_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MRegularSeasonDetailedResults.csv")
detailed_season_results_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WRegularSeasonDetailedResults.csv")

detailed_tournament_results_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneyDetailedResults.csv")
detailed_tournament_results_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WNCAATourneyDetailedResults.csv")

### Geography

In [6]:
game_cities = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/Cities.csv")
game_cities_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MGameCities.csv")
game_cities_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WGameCities.csv")

### Public Rankings

In [7]:
mens_ranking = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MMasseyOrdinals_thruSeason2024_day128.csv")

### Supplemental Data

In [8]:
conference = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/Conferences.csv")
conference_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MTeamConferences.csv")
conference_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WTeamConferences.csv")
conference_tournaments_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MConferenceTourneyGames.csv")

team_names_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MTeamSpellings.csv",encoding='cp1252')
team_names_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WTeamSpellings.csv",encoding='cp1252')
team_coaches_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MTeamCoaches.csv")

tournament_seed_round_slots_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySeedRoundSlots.csv")

tournament_slots_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MNCAATourneySlots.csv")
tournament_slots_women = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/WNCAATourneySlots.csv")

secondary_tournament_teams_men = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MSecondaryTourneyTeams.csv")
secondary_tournaments_results_men= pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/MSecondaryTourneyCompactResults.csv",encoding='cp1252')

sample_submission = pd.read_csv("/kaggle/input/march-machine-learning-mania-2024/sample_submission.csv")

In [9]:
# Calculate win rates and score margins for men
season_results_men['win'] = np.ones(len(season_results_men))
win_rates_men = season_results_men.groupby('WTeamID').size() / (season_results_men.groupby('WTeamID').size() + season_results_men.groupby('LTeamID').size())
win_rates_men = win_rates_men.reset_index(name='WinRate')

season_results_men['ScoreMargin'] = season_results_men['WScore'] - season_results_men['LScore']
avg_score_margin_men = season_results_men.groupby('WTeamID')['ScoreMargin'].mean().reset_index(name='AvgScoreMargin')

# Merge features with team IDs in tournament results for winning and losing teams
tournament_results_men = tournament_results_men.merge(win_rates_men, left_on='WTeamID', right_on='WTeamID', how='left')
tournament_results_men = tournament_results_men.merge(avg_score_margin_men, left_on='WTeamID', right_on='WTeamID', how='left')

# Prepare a combined dataset that includes both perspectives (winning and losing)
tournament_results_men['Outcome'] = 1  # Winning perspective
tournament_results_men_loss = tournament_results_men.copy()
tournament_results_men_loss['WTeamID'], tournament_results_men_loss['LTeamID'] = tournament_results_men_loss['LTeamID'], tournament_results_men_loss['WTeamID']
tournament_results_men_loss['Outcome'] = 0  # Losing perspective

In [10]:
combined_data = pd.concat([tournament_results_men, tournament_results_men_loss])
combined_data.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WinRate,AvgScoreMargin,Outcome
0,1985,136,1116,63,1234,54,N,0,0.640032,16.326608,1
1,1985,136,1120,59,1345,58,N,0,0.548739,13.891271,1
2,1985,136,1207,68,1250,43,N,0,0.605646,14.690678,1
3,1985,136,1229,58,1425,55,N,0,0.534728,10.755869,1
4,1985,136,1242,49,1325,38,N,0,0.805444,16.736581,1


In [11]:
combined_data['Season'].unique()

array([1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995,
       1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006,
       2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017,
       2018, 2019, 2021, 2022, 2023])

In [12]:
season_2023 = combined_data[combined_data['Season'] == 2023]
season_2023.head()
ind = season_2023.index

In [16]:
combined_data

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WinRate,AvgScoreMargin,Outcome
0,1985,136,1116,63,1234,54,N,0,0.640032,16.326608,1
1,1985,136,1120,59,1345,58,N,0,0.548739,13.891271,1
2,1985,136,1207,68,1250,43,N,0,0.605646,14.690678,1
3,1985,136,1229,58,1425,55,N,0,0.534728,10.755869,1
4,1985,136,1242,49,1325,38,N,0,0.805444,16.736581,1
...,...,...,...,...,...,...,...,...,...,...,...
2446,2023,146,1400,88,1274,81,N,0,0.555365,13.088099,0
2447,2023,146,1166,57,1361,56,N,0,0.558040,12.463790,0
2448,2023,152,1274,72,1163,59,N,0,0.682708,16.130227,0
2449,2023,152,1194,72,1361,71,N,0,0.558040,12.463790,0


In [20]:
combined_data.dtypes

Season              int64
DayNum              int64
WTeamID             int64
WScore              int64
LTeamID             int64
LScore              int64
WLoc               object
NumOT               int64
WinRate           float64
AvgScoreMargin    float64
Outcome             int64
dtype: object

In [22]:
# features = ['WinRate', 'AvgScoreMargin']
X = combined_data.drop(['Outcome', 'WLoc'], axis=1)
y = combined_data['Outcome']

#  Check for constant features
# constant_features = [feat for feat in features if np.var(X[feat]) == 0]
# if constant_features:
#     print("Constant features found: ", constant_features)
#     X = X.drop(columns=constant_features)  # Drop constant features

In [23]:
# Train-test split
X_train = X.drop(ind)
X_test = X.loc[ind]
y_train = y.drop(ind)
y_test = y.loc[ind]

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [24]:
# Logistic Regression
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train_scaled, y_train)
y_pred_log_reg = log_reg_model.predict(X_test_scaled)
print("Logistic Regression - Accuracy:", accuracy_score(y_test, y_pred_log_reg))

Logistic Regression - Accuracy: 0.5970149253731343


In [25]:
# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
print("Random Forest - Accuracy:", accuracy_score(y_test, y_pred_rf))

Random Forest - Accuracy: 0.8955223880597015


In [26]:
# Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
print("Gradient Boosting - Accuracy:", accuracy_score(y_test, y_pred_gb))

Gradient Boosting - Accuracy: 0.7910447761194029


In [27]:
# KNN
knn_model = KNeighborsClassifier(n_neighbors=4)
knn_model.fit(X_train_scaled, y_train)
y_pred_knn = knn_model.predict(X_test_scaled)
print("K-NN - Accuracy:", accuracy_score(y_test, y_pred_knn))

K-NN - Accuracy: 0.5


combined_data.to_csv('combined_data.csv', index=False)